# Tensor turbulence flow
In this notebook I recompute some results of \cite{LarK2017} and play with tensors.

#### Why are tensors interesting for me ?
Storing and handling flow data can be challenging especially in the purpose of control and optimization. 
Data can depend on spatial and temporal dimensions as well as special parameters dependent on the optimization problem.


    

In [ ]:
import os
import sys
import numpy as np
from numpy import *
from numpy.linalg import norm
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive
import ipywidgets as widgets
import torch

_repo_root = os.getcwd()
if not os.path.isdir(os.path.join(_repo_root, "torchTT")):
    _repo_root = os.path.abspath(os.path.join(_repo_root, ".."))
_torchtt_path = os.path.join(_repo_root, "torchTT")
if _torchtt_path not in sys.path:
    sys.path.insert(0, _torchtt_path)

import torchtt


def tt_datasize(tt):
    return sum(int(c.numel()) for c in tt.cores)


### 1. Simple 1D sinusoidal example

The first example is 1 dimensional sinusoidal function:

\begin{equation}\label{eq:0Dfunction}
    f_1(x)=3.125\sin(x)\sin(2x)\sin(4x) \qquad\qquad\quad x \in [0,2\pi]
\end{equation}

In [ ]:

Nx = int(2**14) # spatial domain
Lx = 2*pi # domain size

x = linspace(0, Lx, Nx) # spatial grid point

# funnction values
f1 = 3.125 * sin(x) * sin(2*x) * sin(4*x)

plt.plot(x,f1)
plt.xlabel("$x$")
plt.ylabel("$f_1(x)$")
plt.xticks([0,pi,2*pi],("0","$\pi$","$2\pi$"))



In [ ]:
g = f1.reshape(2,2,-1)

f, ax = plt.subplots(1,2)
for i in range(2):
    for j in range(2):
        ax[0].plot(g[i,j], label=str((i,j)))
ax[0].legend()

# note that g[i,j,k] == f1[Nx/2*i + Nx/4*j + k]
assert np.all(f1[:Nx//4] == g[0,0])
ax[1].plot(f1)

This function is stored with $N_x=2^{14}$ grid points (lattice spacing $\Delta x$) in an array:
\begin{equation*}
     X_1[n]=f_1(n\Delta x) \qquad\qquad n=1\dots N_x
\end{equation*}
resulting in $\mathcal{O}(N_x)$ storage.
Lets reshape into a Tensor of order 3:



In [ ]:
Array = np.reshape(f1, (2, 2, int(Nx/4)))

# cast to tensor
T1 = torch.tensor(Array, dtype=torch.float64)

# make a Tensor Train out of the normal Tensor
T1_TT = torchtt.TT(T1)

# what is the error in full Format
T1_TT = T1_TT.round(1e-4)
T1_tilde = T1_TT.full()
print("TTrank", T1_TT.R)
print("Rel_err =", torch.linalg.norm(T1 - T1_tilde) / torch.linalg.norm(T1))
print("storage compression:", tt_datasize(T1_TT) / Nx, "
")

# construct its inner ranks via HOSVD (rounding)
T1_TT = T1_TT.round(1e-3)
T1_tilde = T1_TT.full()

# and output the error of the low-rank approximation
print("TTrank", T1_TT.R)
print("Rel_err =", torch.linalg.norm(T1 - T1_tilde) / torch.linalg.norm(T1))
print("storage compression:", tt_datasize(T1_TT) / Nx)


In [ ]:
# torchTT does not provide graph drawing; show core shapes instead.
print([c.shape for c in T1_TT.cores])


The 3 order Tensor $T_1[n_1,n_2,n_3]$ is approximated \textbf{"exactly"}  for TTrank $\ge 2$ (cited from section 3.1 at p. 4) by:

\begin{equation}\label{eq:T3}
    T_1[n_1,n_2,n_3] \approx \sum_{i=1,j=1,2}U_1[n_1,i]U_2[i,n_2,j]U_3[j,n3]
\end{equation}
This is not true as we see from the relative error in the Frobenius norm above. Only for TTrank = 4 the Tensor is \textit{"approximated exactly"} (rel err $\approx 10^{-15}$). 

Let's have a look at the matrices in the TTrain:
\begin{align*}
    U_1 &= \frac{1}{\sqrt{2}}
        \begin{pmatrix}
            -1 & 1
        \end{pmatrix} &
    U_2 &= \frac{1}{\sqrt{2}}
        \begin{pmatrix}
            -1 & 1 \\
             1 & 1
        \end{pmatrix}&
     U_3 &= \begin{pmatrix}
            * & \cdots & * \\
            * & \cdots & * 
            \end{pmatrix} \in \mathbb{R}^{2\times N_x/4}
\end{align*}


In [ ]:
# calculate components of the Tensor
U1 = T1_TT.cores[0]
U2 = T1_TT.cores[1]
U3 = T1_TT.cores[2]

print("U_1 = ", U1.detach().cpu().numpy(),"
")
print("U_2 = ",U2.detach().cpu().numpy(),"
")

fnew = U3.detach().cpu().numpy()
for i in range(size(fnew,0)):
    plt.plot(fnew[i,:])

plt.title('$U_3$')
plt.tick_params(labelbottom=False)
plt.xlabel('$x$')
plt.legend(("1. row","2. row"))
ap = U3.detach().cpu().numpy()


In [ ]:
def TTapprox_fun(fun, order, TTrank, threshold=1e-14, orig=None):
    # Approximate fun with a tensor train of a certain order.
    Nx = size(fun, 0)  # Number of grid points
    order = order - 1

    # Reshape array to tensor dimensions
    A = reshape(fun, np.append([2] * order, int(Nx / 2**order)))
    TTrain = torchtt.TT(torch.tensor(A, dtype=torch.float64), eps=threshold, rmax=TTrank)

    # compare Tensortrain with original data
    Ttilde = TTrain.full().detach().cpu().numpy()
    fun_tilde = reshape(Ttilde, -1)

    # calculate relative error
    if orig is not None:
        err_euclid = norm(fun_tilde - orig) / norm(orig)
    else:
        err_euclid = norm(fun_tilde - fun) / norm(fun)
    err_frob = np.linalg.norm(A - Ttilde) / np.linalg.norm(A)
    compress_factor = tt_datasize(TTrain) / Nx

    return err_euclid, compress_factor, TTrain, fun_tilde, err_frob


In [ ]:
def plot_sinusoid(fun,TTrank=2,order=3):
    error, compress_factor, TTrain, fun_tilde, err_frob= TTapprox_fun(fun,order,TTrank,1e-14)
    plt.figure()
    plt.plot(x,fun_tilde,'-')
    plt.plot(x,fun,':')
    plt.legend(("TT-rank ="+str(max(TTrain.R))+" approximation","$f_1(x)$"))
    plt.xlabel("$x$")
    plt.ylabel("$f_1(x)$")
    plt.grid()

    print("compression factor=",compress_factor)
    print(TTrain.R)
    print("relative error = ",error*100 ,"%")


plot_sinusoid(f1,2,3)


We now perform a parameter study of the order of the TTrain and the TTrank to see how storage and relative error behave. Suprisingly the order 2 Tensor with TT-rank 2 is already quite good.

In [ ]:
ord_list=range(2,14)
rank_list=range(1,15)

M = size(ord_list)
N = size(rank_list)
errors = zeros([M,N])
compress = zeros([M,N])
TTapprox_fun(f1,3,1, 1e-14)


for j,rank in enumerate(rank_list):
    for i,order in enumerate(ord_list):
        res = TTapprox_fun(f1,order,rank, 1e-14)
        errors[i,j]=res[0]
        compress[i,j]=res[1]

        
plt.figure()
plt.subplot(211)
for e,err in enumerate(errors.T[::2]):
    plt.semilogy(ord_list, err, label="TTrank=%d"%rank_list[2*e])
    
# plt.semilogy(ord_list,errors[:,0],'x--')
# plt.semilogy(ord_list,errors[:,1],'o-')
# plt.semilogy(ord_list,errors[:,3],':+')
# plt.plot(ord_list,errors[:,-1],':*')
plt.legend() # (['TTrank='+str(rank_list[0]),"TTrank="+str(rank_list[1]),"TTrank="+str(rank_list[3]),"TTrank="+str(rank_list[-1])])
plt.xlabel("order")
plt.ylabel("rel error")
plt.grid()

plt.subplot(212)

for e,cprs in enumerate(compress.T[::2]):
    plt.semilogy(ord_list, cprs, label="TTrank=%d"%rank_list[2*e])
    
# plt.plot(ord_list,compress[:,0],'x--')
# plt.plot(ord_list,compress[:,1],'o-')
# plt.plot(ord_list,compress[:,3],':+')
# plt.plot(ord_list,compress[:,-1],':*')
plt.legend()
plt.grid()
# plt.legend(['TTrank='+str(rank_list[0]),"TTrank="+str(rank_list[1]),"TTrank="+str(rank_list[3]),"TTrank="+str(rank_list[-1])])
plt.xlabel("order")
plt.ylabel("compression rate")
# plt.yscale("log")

plt.show()

### 2. Detection of self similar structures with noise

\begin{definition}[Self similar]\label{def:self_sim}
   $A$ is a self-similar set, if it is the invariant set (attractor) of an Iterated function system {$f_i, i=1\dots,m$}:
   
   \begin{align*}
     A={\bigcup _{i=1}^m} f_i(A)
    \end{align*}

\end{definition}


Construct a self similar function from $g(x)=$rect$(x-7)$:

\begin{equation}\label{eq:sel}
    f(x)=2000\sum_{k=0}^N\frac{g(x/(1.5)^k)}{(1.5)^k}
\end{equation}


In [ ]:
# Construct Data series
Nx= 2**14
x = linspace(0,8,Nx)

def rect(x):
    return heaviside(x-6.5,1)-heaviside(x-7.5,1)

data = 0*x
for k in range(0,10):
    data+=rect(x*1.5**k)/1.5**k

data*=2000

plt.plot(x,data)
plt.plot(x,2000*rect(x))
plt.legend(("f(x)",'$\sim$rect$(x)$'))
plt.ylabel("$f(x)$")
plt.xlabel("x")


Modulate a random noise on the data:


In [ ]:
data1 = data + (1-2*np.random.rand(size(data)))
data10 = data + 100*(1-2*np.random.rand(size(data)))
plt.plot(data)
plt.plot(data1,"*")
plt.plot(data10,'+')

In [ ]:
order=13
rank_list=range(1,150)
err_eucl=np.zeros([3,size(rank_list)])
compress=np.zeros([3,size(rank_list)])
err_frob=np.zeros([3,size(rank_list)])
ftilde=[]

for i,rank in enumerate(rank_list):
    
    # results for no noise
    res = TTapprox_fun(data,order,rank, 1e-14)
    err_eucl[0,i]=res[0]
    err_frob[0,i]=res[-1]
    compress[0,i]=res[1]
        
    # noise amplitude 1
    res = TTapprox_fun(data1,order,rank, 1e-14, data)  # self similarity --> noise is large in small strctures and cannot be cut off by HSVD
    err_eucl[1,i]=res[0]
    err_frob[1,i]=res[-1]
    compress[1,i]=res[1]
        
    #noise amplitude 10
    res = TTapprox_fun(data10,order,rank, 1e-14, data)
    err_eucl[2,i]=res[0]
    err_frob[2,i]=res[-1]
    compress[2,i]=res[1]
    
plt.semilogy(rank_list,err_eucl[0,:])
plt.semilogy(rank_list,err_eucl[1,:])
plt.semilogy(rank_list,err_eucl[2,:])
plt.xlabel("TTrank")
plt.ylabel("rel euclidean error")
plt.legend(["original","noise amplitude 1","noise amplitude 10"])

plt.figure()
for i in range(size(compress,0)):
    plt.loglog(err_frob[i,:],compress[i,:])
plt.legend(["original","noise amplitude 1","noise amplitude 10"])
plt.xlabel("rel frob norm")
plt.ylabel("compression factor")

In [ ]:
# plot results for TTrank=2 and 4

res = TTapprox_fun(data10,order,2, 1e-14)
plt.plot(x,data10)
plt.plot(x,res[3],':')
res = TTapprox_fun(data10,order,4, 1e-14)
plt.plot(x,res[3],'--')
res = TTapprox_fun(data10,order,10, 1e-14)
plt.plot(x,res[3],'--')
plt.legend(["original","rank 2","rank 4","rank 10"])


# References

(<a id="cit-LarK2017" href="#call-LarK2017">von Larcher and Klein, 2017</a>) von Larcher Thomas and Klein Rupert, ``_On identification of self-similar characteristics using the Tensor Train decomposition method with application to channel turbulence flow_'', arXiv preprint arXiv:1708.07780, vol. , number , pp. ,  2017.

